In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

In [3]:
# Open the original CSV as it was downloaded from MYCV
original = pd.read_csv('yannis_scholarly.csv')
original.head()

,ID,Scholarly & creative work type,Reporting date 1,Reporting date 2,Abstract OR Description,Acceptance date OR Date of acceptance,Addresses,Altmetric attention score,File(s) confidential,Associated authors,...,Citation count,Indexed in DOAJ,DOAJ CC Licence,Files in OpenBU RT2,OpenBU RT2 public URL,Deposited to OpenBU RT2,Item status in OpenBU RT2,Proprietary ID,Source,Canonical journal title
0,29972,Scholarly article,01/08/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,NaN,NaN,WOS:000309996500040,Web of Science,IEEE Transactions on Power Systems
1,29973,Scholarly article,01/12/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,NaN,NaN,10.1109/TMC.2011.214,Crossref,IEEE Transactions on Mobile Computing
2,30260,Scholarly article,01/01/2012,NaN,NaN,NaN,"St. Paul, Minnesota",NaN,NaN,NaN,...,NaN,No,NaN,1.0,https://hdl.handle.net/2144/18026,Yes,Live,2CC8EA0E-5D44-4C73-90A8-B996799539CB,Manual,IEEE International Conference on Robotics and ...
3,30488,Scholarly article,01/06/2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,NaN,NaN,10.1109/TNET.2012.2216897,Crossref,IEEE ACM Transactions on Networking
4,30489,Scholarly article,01/08/2012,NaN,<jats:p> We consider the maximum lifetime rout...,NaN,NaN,NaN,NaN,NaN,...,NaN,Yes,CC BY,NaN,NaN,NaN,NaN,10.1155/2012/523787,Crossref,International Journal of Distributed Sensor Ne...


In [4]:
# Grab the columns that are most likely to be of use and convert them to the correct types
columns_of_interest = {
    'Reporting date 1': 'string',
    'Scholarly & creative work type': 'string',
    'Authors OR Patent owners OR Presenters': 'string',
    'URL OR Author URL': 'string',
    'DOI': 'string',
    'Funding': 'string',
    'Published proceedings OR Journal': 'string',
    'Conference name OR Presented at OR Meeting or conference': 'string',
    'Status': 'string',
    'Publisher': 'string',
    'Publication date OR Date awarded OR Presentation date': 'datetime64',
    'Title OR Chapter title': 'string',
    'Sub types': 'string',
    'Canonical journal title': 'string'
}
data = original[columns_of_interest.keys()]
data = data.astype(columns_of_interest)
data['Reporting date 1'] = pd.to_datetime(data['Reporting date 1'], format='%d/%m/%Y')
data.head()

,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title
0,2012-08-01,Scholarly article,"Paschalidis IC, Li B, Caramanis MC",http://gateway.webofknowledge.com/gateway/Gate...,10.1109/TPWRS.2012.2183007,"NSF, NSF, DOE, ARO, ODDR&E MURI10 program",IEEE TRANSACTIONS ON POWER SYSTEMS,<NA>,Published,IEEE-INST ELECTRICAL ELECTRONICS ENGINEERS INC,2012-08-01,Demand-Side Management for Regulation Service ...,"Article, Journal",IEEE Transactions on Power Systems
1,2012-12-01,Scholarly article,"Li K, Guo D, Lin Y, Paschalidis IC",<NA>,10.1109/tmc.2011.214,<NA>,IEEE Transactions on Mobile Computing,<NA>,Published,Institute of Electrical and Electronics Engine...,2012-12-01,Position and Movement Detection of Wireless Se...,<NA>,IEEE Transactions on Mobile Computing
2,2012-01-01,Scholarly article,"Ding X-C, Wang J, Lahijanian M, Paschalidis IC...",<NA>,<NA>,<NA>,Proceedings of IEEE International Conference o...,<NA>,<NA>,<NA>,NaT,Temporal Logic Motion Control using Actor-Crit...,inproceedings,IEEE International Conference on Robotics and ...
3,2013-06-01,Scholarly article,"Moazzez-Estanjini R, Wang J, Paschalidis IC",<NA>,10.1109/tnet.2012.2216897,<NA>,IEEE/ACM Transactions on Networking,<NA>,Published,Institute of Electrical and Electronics Engine...,2013-06-01,Scheduling Mobile Nodes for Cooperative Data T...,<NA>,IEEE ACM Transactions on Networking
4,2012-08-01,Scholarly article,"Paschalidis IC, Wu R",<NA>,10.1155/2012/523787,<NA>,International Journal of Distributed Sensor Ne...,<NA>,Published,SAGE Publications,2012-08-01,Robust Maximum Lifetime Routing and Energy All...,<NA>,International Journal of Distributed Sensor Ne...


In [20]:
year = 2021
frp_title = 'Leveraging AI to Examine Disparities and Bias in Health Care'
request_url = 'https://ollama-sail-24887a.apps.shift.nerc.mghpcc.org/api/generate'

In [21]:
publications = data[data['Reporting date 1'].dt.year >= year]
publications.head()

,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title
296,2021-02-02,Conference paper (Published),"Queeney J, Paschalidis I, Cassandras C",https://ojs.aaai.org/index.php/AAAI/article/view/17130,<NA>,<NA>,<NA>,35th AAAI Conference on Artificial Intelligence (AAAI-21),Published,<NA>,2021-02-02,"Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region Approach",<NA>,Proceedings of the ... AAAI Conference on Artificial Intelligence. AAAI Conference on Artificial Intelligence
297,2021-02-01,Conference paper (Published),"Zhang Y, Wilson D, Paschalidis I, Coskun A",<NA>,10.23919/DATE51398.2021.9474075,<NA>,<NA>,"Design, Automation and Test in Europe Conference (DATE 2021)",Published,<NA>,2021-02-01,A Data Center Demand Response Policy for Real-World Workload Scenarios in HPC,<NA>,<NA>
298,2021-04-13,Conference paper (Published),"Zhang Z, Paschalidis I",http://proceedings.mlr.press/v130/zhang21b/zhang21b.pdf,<NA>,<NA>,<NA>,24th International Conference on Artificial Intelligence and Statistics (AISTATS),Accepted,<NA>,2021-04-13,Provable Hierarchical Imitation Learning via EM,<NA>,<NA>
304,2021-01-01,Scholarly article,"Sotudian S, Desta IT, Hashemi N, Zarbafian S, Kozakov D, Vakili P, Vajda S, Paschalidis IC",https://www.ncbi.nlm.nih.gov/pubmed/33995918,10.1016/j.csbj.2021.04.028,"NIGMS NIH HHS, NIGMS NIH HHS, NIGMS NIH HHS",Comput Struct Biotechnol J,<NA>,Published online,<NA>,2021-01-01,Improved cluster ranking in protein-protein docking using a regression approach.,Journal Article,Computational and Structural Biotechnology Journal
305,2021-09-01,Conference paper (Published),"Wollenstein-Betech S, Paschalidis IC, Cassandras CG",http://arxiv.org/abs/2107.06937v1,10.1109/ITSC48978.2021.9564808,<NA>,<NA>,2021 IEEE International Intelligent Transportation Systems Conference,Published,<NA>,2021-09-01,Planning Strategies for Lane Reversals in Transportation Networks,<NA>,<NA>


In [22]:
publications['Status'].value_counts(dropna=False)

<NA>                52
Published           41
Published online    16
Accepted             5
Submitted            1
Name: Status, dtype: Int64

In [23]:
pd.set_option('display.max_colwidth', None)
publications['Title OR Chapter title']

296                                 Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region   Approach
297                                    A Data Center Demand Response Policy for Real-World Workload Scenarios in  HPC
298                                                                   Provable Hierarchical Imitation Learning via EM
304                                  Improved cluster ranking in protein-protein docking using a regression approach.
305                                                 Planning Strategies for Lane Reversals in Transportation Networks
                                                            ...                                                      
412                                                                            Safe and Robust Reinforcement Learning
413                                    AI Models and the Role of Sociodemographic Factors in Disease and its Severity
414                                                     

In [65]:
prompt = '''
<<SYS>>
You are an assistant tasked with classifying whether the given publication title is associated with the given research topic.

Specifically, the content should be marked as relevant if it involves: 
    1. Publications which are likely to have been written based on the research topic as a prompt.
    2. If the publication title has overlap with the research topic.

Generate a short response indicating whether the content meets any of the above criteria. Respond with "Yes" for relevance or "No" if the publication does not have high overlap.
<</SYS>> 

[INST]Assess the given headline and article body based on the specified criteria. Provide a concise response indicating relevance.

Publication Title: {publication_title} 

Research Topic: {frp_title} \n\n [/INST]
'''

request_data = {
    'raw': True,
    'stream': False
}

In [84]:
def is_part_of_frp(publication_title: str, frp_title: str, model: str):
    request_data['prompt'] = prompt.format(frp_title=frp_title, publication_title=publication_title)
    request_data['model'] = model
    response = requests.post(request_url, json=request_data)
    response_data = response.json()['response'].lower()
    # print(response_data)
    if 'yes' in response_data:
        return True
    elif 'no' in response_data:
        return False
    return pd.NA
is_part_of_frp('Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region   Approach', frp_title, 'mistral:7b')

based on the given criteria, it appears that the publication title is not directly related to the research topic. the publication title discusses "uncertainty-aware policy optimization" which is a method used in reinforce learning, while the research topic focuses on "leveraging ai to examine disparities and bias in health care".

therefore, my assessment would be "no", as there does not seem to be a high degree of overlap between the two.


False

# LLAMA2:13B Testing

In [75]:
llama_13_results = publications.copy()
llama_13_results['Part of FRP'] = publications['Title OR Chapter title'].apply(lambda title: is_part_of_frp(title, frp_title, 'llama2:13b'))

In [76]:
llama_13_results.head()

,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title,Part of FRP
296,2021-02-02,Conference paper (Published),"Queeney J, Paschalidis I, Cassandras C",https://ojs.aaai.org/index.php/AAAI/article/view/17130,<NA>,<NA>,<NA>,35th AAAI Conference on Artificial Intelligence (AAAI-21),Published,<NA>,2021-02-02,"Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region Approach",<NA>,Proceedings of the ... AAAI Conference on Artificial Intelligence. AAAI Conference on Artificial Intelligence,True
297,2021-02-01,Conference paper (Published),"Zhang Y, Wilson D, Paschalidis I, Coskun A",<NA>,10.23919/DATE51398.2021.9474075,<NA>,<NA>,"Design, Automation and Test in Europe Conference (DATE 2021)",Published,<NA>,2021-02-01,A Data Center Demand Response Policy for Real-World Workload Scenarios in HPC,<NA>,<NA>,False
298,2021-04-13,Conference paper (Published),"Zhang Z, Paschalidis I",http://proceedings.mlr.press/v130/zhang21b/zhang21b.pdf,<NA>,<NA>,<NA>,24th International Conference on Artificial Intelligence and Statistics (AISTATS),Accepted,<NA>,2021-04-13,Provable Hierarchical Imitation Learning via EM,<NA>,<NA>,False
304,2021-01-01,Scholarly article,"Sotudian S, Desta IT, Hashemi N, Zarbafian S, Kozakov D, Vakili P, Vajda S, Paschalidis IC",https://www.ncbi.nlm.nih.gov/pubmed/33995918,10.1016/j.csbj.2021.04.028,"NIGMS NIH HHS, NIGMS NIH HHS, NIGMS NIH HHS",Comput Struct Biotechnol J,<NA>,Published online,<NA>,2021-01-01,Improved cluster ranking in protein-protein docking using a regression approach.,Journal Article,Computational and Structural Biotechnology Journal,True
305,2021-09-01,Conference paper (Published),"Wollenstein-Betech S, Paschalidis IC, Cassandras CG",http://arxiv.org/abs/2107.06937v1,10.1109/ITSC48978.2021.9564808,<NA>,<NA>,2021 IEEE International Intelligent Transportation Systems Conference,Published,<NA>,2021-09-01,Planning Strategies for Lane Reversals in Transportation Networks,<NA>,<NA>,True


In [77]:
llama_13_results['Part of FRP'].value_counts(dropna=False)

True     76
False    39
Name: Part of FRP, dtype: int64

In [78]:
llama_13_results.to_csv('llama13_publications.csv')

# Mistral:7B Testing

In [80]:
mistral_7_results = publications.copy()
mistral_7_results['Part of FRP'] = publications['Title OR Chapter title'].apply(lambda title: is_part_of_frp(title, frp_title, 'mistral:7b'))

In [81]:
mistral_7_results.head()

,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title,Part of FRP
296,2021-02-02,Conference paper (Published),"Queeney J, Paschalidis I, Cassandras C",https://ojs.aaai.org/index.php/AAAI/article/view/17130,<NA>,<NA>,<NA>,35th AAAI Conference on Artificial Intelligence (AAAI-21),Published,<NA>,2021-02-02,"Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region Approach",<NA>,Proceedings of the ... AAAI Conference on Artificial Intelligence. AAAI Conference on Artificial Intelligence,False
297,2021-02-01,Conference paper (Published),"Zhang Y, Wilson D, Paschalidis I, Coskun A",<NA>,10.23919/DATE51398.2021.9474075,<NA>,<NA>,"Design, Automation and Test in Europe Conference (DATE 2021)",Published,<NA>,2021-02-01,A Data Center Demand Response Policy for Real-World Workload Scenarios in HPC,<NA>,<NA>,False
298,2021-04-13,Conference paper (Published),"Zhang Z, Paschalidis I",http://proceedings.mlr.press/v130/zhang21b/zhang21b.pdf,<NA>,<NA>,<NA>,24th International Conference on Artificial Intelligence and Statistics (AISTATS),Accepted,<NA>,2021-04-13,Provable Hierarchical Imitation Learning via EM,<NA>,<NA>,False
304,2021-01-01,Scholarly article,"Sotudian S, Desta IT, Hashemi N, Zarbafian S, Kozakov D, Vakili P, Vajda S, Paschalidis IC",https://www.ncbi.nlm.nih.gov/pubmed/33995918,10.1016/j.csbj.2021.04.028,"NIGMS NIH HHS, NIGMS NIH HHS, NIGMS NIH HHS",Comput Struct Biotechnol J,<NA>,Published online,<NA>,2021-01-01,Improved cluster ranking in protein-protein docking using a regression approach.,Journal Article,Computational and Structural Biotechnology Journal,False
305,2021-09-01,Conference paper (Published),"Wollenstein-Betech S, Paschalidis IC, Cassandras CG",http://arxiv.org/abs/2107.06937v1,10.1109/ITSC48978.2021.9564808,<NA>,<NA>,2021 IEEE International Intelligent Transportation Systems Conference,Published,<NA>,2021-09-01,Planning Strategies for Lane Reversals in Transportation Networks,<NA>,<NA>,False


In [82]:
mistral_7_results['Part of FRP'].value_counts(dropna=False)

False    83
True     25
<NA>      7
Name: Part of FRP, dtype: int64

In [83]:
mistral_7_results.to_csv('mistral7_publications.csv')

# ChatGPT 3.5 Turbo Testing

In [85]:
import openai
import os

In [96]:
client = openai.OpenAI(api_key=os.environ["OPENAI_KEY"])

In [97]:
chatgpt_prompt = '''
You are an assistant tasked with classifying whether the given publication title is associated with the given research topic.

Specifically, the content should be marked as relevant if it involves: 
    1. Publications which are likely to have been written based on the research topic as a prompt.
    2. If the publication title has overlap with the research topic.

Generate a short response indicating whether the content meets any of the above criteria. Respond with "Yes" for relevance or "No" if the publication does not have high overlap.
'''
request_prompt = '''
Assess the given headline and article body based on the specified criteria. Provide a concise response indicating relevance.

Publication Title: {publication_title} 

Research Topic: {frp_title}
'''

In [98]:
chat_completion = client.chat.completions.create(
    messages = [
        {
            "role": "system",
            "content": chatgpt_prompt
        },
        {
            "role": "user",
            "content": request_prompt.format(frp_title=frp_title, publication_title='Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region   Approach')
        }
    ],
    model="gpt-3.5-turbo"
)

In [104]:
'yes' in chat_completion.choices[0].message.content.lower()

True

In [107]:
def chatgpt_is_part_of_frp(publication_title: str, frp_title: str):
    chat_completion = client.chat.completions.create(
        messages = [
            {
                "role": "system",
                "content": chatgpt_prompt
            },
            {
                "role": "user",
                "content": request_prompt.format(frp_title=frp_title, publication_title='Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region   Approach')
            }
        ],
        model="gpt-3.5-turbo"
    )
    
    if 'yes' in chat_completion.choices[0].message.content.lower():
        return True
    elif 'no' in chat_completion.choices[0].message.content.lower():
        return False
    return pd.NA
chatgpt_is_part_of_frp('Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region   Approach', frp_title)

True

In [108]:
chatgpt_35_results = publications.copy()
chatgpt_35_results['Part of FRP'] = publications['Title OR Chapter title'].apply(lambda title: chatgpt_is_part_of_frp(title, frp_title))

In [109]:
chatgpt_35_results.head()

,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title,Part of FRP
296,2021-02-02,Conference paper (Published),"Queeney J, Paschalidis I, Cassandras C",https://ojs.aaai.org/index.php/AAAI/article/view/17130,<NA>,<NA>,<NA>,35th AAAI Conference on Artificial Intelligence (AAAI-21),Published,<NA>,2021-02-02,"Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region Approach",<NA>,Proceedings of the ... AAAI Conference on Artificial Intelligence. AAAI Conference on Artificial Intelligence,True
297,2021-02-01,Conference paper (Published),"Zhang Y, Wilson D, Paschalidis I, Coskun A",<NA>,10.23919/DATE51398.2021.9474075,<NA>,<NA>,"Design, Automation and Test in Europe Conference (DATE 2021)",Published,<NA>,2021-02-01,A Data Center Demand Response Policy for Real-World Workload Scenarios in HPC,<NA>,<NA>,True
298,2021-04-13,Conference paper (Published),"Zhang Z, Paschalidis I",http://proceedings.mlr.press/v130/zhang21b/zhang21b.pdf,<NA>,<NA>,<NA>,24th International Conference on Artificial Intelligence and Statistics (AISTATS),Accepted,<NA>,2021-04-13,Provable Hierarchical Imitation Learning via EM,<NA>,<NA>,True
304,2021-01-01,Scholarly article,"Sotudian S, Desta IT, Hashemi N, Zarbafian S, Kozakov D, Vakili P, Vajda S, Paschalidis IC",https://www.ncbi.nlm.nih.gov/pubmed/33995918,10.1016/j.csbj.2021.04.028,"NIGMS NIH HHS, NIGMS NIH HHS, NIGMS NIH HHS",Comput Struct Biotechnol J,<NA>,Published online,<NA>,2021-01-01,Improved cluster ranking in protein-protein docking using a regression approach.,Journal Article,Computational and Structural Biotechnology Journal,True
305,2021-09-01,Conference paper (Published),"Wollenstein-Betech S, Paschalidis IC, Cassandras CG",http://arxiv.org/abs/2107.06937v1,10.1109/ITSC48978.2021.9564808,<NA>,<NA>,2021 IEEE International Intelligent Transportation Systems Conference,Published,<NA>,2021-09-01,Planning Strategies for Lane Reversals in Transportation Networks,<NA>,<NA>,True


In [112]:
chatgpt_35_results['Part of FRP'].value_counts(dropna=False)

True     105
False     10
Name: Part of FRP, dtype: int64

In [113]:
chatgpt_35_results.to_csv('chatgpt35_publications.csv')

# Agreement Overlap

In [5]:
llama2 = pd.read_csv('llama13_publications.csv') #.set_index('Title OR Chapter title')
mistral = pd.read_csv('mistral7_publications.csv')# .set_index('Title OR Chapter title')
chatgpt = pd.read_csv('chatgpt35_publications.csv')#.set_index('Title OR Chapter title')

In [6]:
merged = llama2.merge(mistral[['Title OR Chapter title', 'Part of FRP']], on='Title OR Chapter title', how='inner', suffixes=('_llama', '_mistral'))
merged = merged.merge(chatgpt[['Title OR Chapter title', 'Part of FRP']], on='Title OR Chapter title', how='inner', suffixes=('', '_chatgpt'))
merged = merged.rename(columns={'Part of FRP': 'Part of FRP_chatgpt'})
merged.head()

,Unnamed: 0,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title,Part of FRP_llama,Part of FRP_mistral,Part of FRP_chatgpt
0,296,2021-02-02,Conference paper (Published),"Queeney J, Paschalidis I, Cassandras C",https://ojs.aaai.org/index.php/AAAI/article/vi...,NaN,NaN,NaN,35th AAAI Conference on Artificial Intelligenc...,Published,NaN,2021-02-02,Uncertainty-Aware Policy Optimization: A Robus...,NaN,Proceedings of the ... AAAI Conference on Arti...,True,False,True
1,297,2021-02-01,Conference paper (Published),"Zhang Y, Wilson D, Paschalidis I, Coskun A",NaN,10.23919/DATE51398.2021.9474075,NaN,NaN,"Design, Automation and Test in Europe Conferen...",Published,NaN,2021-02-01,A Data Center Demand Response Policy for Real-...,NaN,NaN,False,False,True
2,298,2021-04-13,Conference paper (Published),"Zhang Z, Paschalidis I",http://proceedings.mlr.press/v130/zhang21b/zha...,NaN,NaN,NaN,24th International Conference on Artificial In...,Accepted,NaN,2021-04-13,Provable Hierarchical Imitation Learning via EM,NaN,NaN,False,False,True
3,304,2021-01-01,Scholarly article,"Sotudian S, Desta IT, Hashemi N, Zarbafian S, ...",https://www.ncbi.nlm.nih.gov/pubmed/33995918,10.1016/j.csbj.2021.04.028,"NIGMS NIH HHS, NIGMS NIH HHS, NIGMS NIH HHS",Comput Struct Biotechnol J,NaN,Published online,NaN,2021-01-01,Improved cluster ranking in protein-protein do...,Journal Article,Computational and Structural Biotechnology Jou...,True,False,True
4,305,2021-09-01,Conference paper (Published),"Wollenstein-Betech S, Paschalidis IC, Cassandr...",http://arxiv.org/abs/2107.06937v1,10.1109/ITSC48978.2021.9564808,NaN,NaN,2021 IEEE International Intelligent Transporta...,Published,NaN,2021-09-01,Planning Strategies for Lane Reversals in Tran...,NaN,NaN,True,False,True


In [7]:
merged['Agree'] = (merged['Part of FRP_llama'] == merged['Part of FRP_chatgpt']) & (merged['Part of FRP_chatgpt'] == merged['Part of FRP_mistral'])
agreed = merged['Agree'].value_counts(dropna=False).reset_index(name='count')
agreed['total'] = len(merged.index)
agreed['percent'] = agreed['count'] / agreed['total'] * 100
agreed

,index,count,total,percent
0,False,145,235,61.702128
1,True,90,235,38.297872


In [9]:
pd.set_option('display.max_colwidth', None)
llama2[llama2['Part of FRP'] == True]

,Unnamed: 0,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title,Part of FRP
0,296,2021-02-02,Conference paper (Published),"Queeney J, Paschalidis I, Cassandras C",https://ojs.aaai.org/index.php/AAAI/article/view/17130,NaN,NaN,NaN,35th AAAI Conference on Artificial Intelligence (AAAI-21),Published,NaN,2021-02-02,"Uncertainty-Aware Policy Optimization: A Robust, Adaptive Trust Region Approach",NaN,Proceedings of the ... AAAI Conference on Artificial Intelligence. AAAI Conference on Artificial Intelligence,True
3,304,2021-01-01,Scholarly article,"Sotudian S, Desta IT, Hashemi N, Zarbafian S, Kozakov D, Vakili P, Vajda S, Paschalidis IC",https://www.ncbi.nlm.nih.gov/pubmed/33995918,10.1016/j.csbj.2021.04.028,"NIGMS NIH HHS, NIGMS NIH HHS, NIGMS NIH HHS",Comput Struct Biotechnol J,NaN,Published online,NaN,2021-01-01,Improved cluster ranking in protein-protein docking using a regression approach.,Journal Article,Computational and Structural Biotechnology Journal,True
4,305,2021-09-01,Conference paper (Published),"Wollenstein-Betech S, Paschalidis IC, Cassandras CG",http://arxiv.org/abs/2107.06937v1,10.1109/ITSC48978.2021.9564808,NaN,NaN,2021 IEEE International Intelligent Transportation Systems Conference,Published,NaN,2021-09-01,Planning Strategies for Lane Reversals in Transportation Networks,NaN,NaN,True
5,306,2021-09-14,Scholarly article,"Amini S, Zhang L, Hao B, Gupta A, Song M, Karjadi C, Lin H, Kolachalama VB, Au R, Paschalidis IC",https://www.ncbi.nlm.nih.gov/pubmed/34334396,10.3233/JAD-210299,"NIGMS NIH HHS, NHLBI NIH HHS, NIA NIH HHS, NIA NIH HHS, NIA NIH HHS, NIA NIH HHS, NIA NIH HHS, NICHD NIH HHS, NIA NIH HHS, NIA NIH HHS, NHLBI NIH HHS, NIA NIH HHS, NHLBI NIH HHS, NIA NIH HHS, NCI NIH HHS, NIA NIH HHS, NHLBI NIH HHS",J Alzheimers Dis,NaN,Published,NaN,2021-01-01,An Artificial Intelligence-Assisted Method for Dementia Detection Using Images from the Clock Drawing Test.,"Journal Article, Research Support, N.I.H., Extramural, Research Support, Non-U.S. Gov't, Research Support, U.S. Gov't, Non-P.H.S.",Journal of Alzheimer's Disease,True
7,308,2021-09-07,Scholarly article,"Zhao Y, Paschalidis IC, Hu J",https://www.ncbi.nlm.nih.gov/pubmed/34493261,10.1186/s12913-021-06886-3,"National Natural Science Foundation of China, National Natural Science Foundation of China, National Science Foundation, National Science Foundation, National Science Foundation, NIGMS NIH HHS, Office of Naval Research, NIH HHS",BMC Health Serv Res,NaN,Published online,NaN,2021-09-07,The impact of payer status on hospital admissions: evidence from an academic medical center,NaN,BMC Health Services Research,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,410,2023-08-22,Scholarly presentation,Paschalidis IC,NaN,NaN,NaN,NaN,Department of Energy Buildings Energy Efficiency Frontiers & Innovation Technologies (BENEFIT) Program,NaN,NaN,2023-08-22,A Distributed Optimization Framework to Control the Overall Load Consumption of a Residential Neighborhood,NaN,NaN,True
110,412,2023-08-09,Scholarly presentation,Paschalidis IC,NaN,NaN,NaN,NaN,ONR Science of Autonomy Workshop,NaN,NaN,2023-08-09,Safe and Robust Reinforcement Learning,NaN,NaN,True
111,413,2023-03-17,Scholarly presentation,Paschalidis IC,NaN,NaN,NaN,NaN,Healthcare Information and Management Systems Society (HIMSS),NaN,NaN,2023-03-17,AI Models and the Role of Sociodemographic Factors in Disease and its Severity,NaN,NaN,True
112,414,2023-03-08,Scholarly presentation,Paschalidis IC,NaN,NaN,NaN,NaN,NSF Workshop on AI-Enabled Scientific Revolution,NaN,NaN,2023-03-08,AI for (Good) Health,NaN,NaN,True


In [10]:
mistral[mistral['Part of FRP'] == True]

,Unnamed: 0,Reporting date 1,Scholarly & creative work type,Authors OR Patent owners OR Presenters,URL OR Author URL,DOI,Funding,Published proceedings OR Journal,Conference name OR Presented at OR Meeting or conference,Status,Publisher,Publication date OR Date awarded OR Presentation date,Title OR Chapter title,Sub types,Canonical journal title,Part of FRP
21,322,2021-07-21,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,INFORMS Healthcare Conference,NaN,NaN,2021-07-21,Racial Disparities Revealed by Models Predicting COVID-19 Outcomes,NaN,NaN,True
22,323,2021-07-26,Scholarly presentation,"Amini S, Zhang L, Hao B, Gupra A, Song M, Karjadi C, Lin H, Kolachalama V, Au R, Paschalidis I",NaN,NaN,NaN,NaN,Alzheimer's Association International Conference,NaN,NaN,2021-07-26,Remote Diagnosis of Dementia using AI methods on clock drawing images,NaN,Alzheimer's and Dementia,True
26,327,2021-03-01,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,Boston University Medical School,NaN,NaN,2021-03-01,Pandemic Predictive Analytics,NaN,NaN,True
27,328,2021-04-23,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,BU Research Computing Governance Meeting,NaN,NaN,2021-04-23,Data Science and Optimization with Applications in Computational Biology and Medicine,NaN,NaN,True
28,329,2021-05-06,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,"Charles DeLisi Award Lecture, Boston University",NaN,NaN,2021-05-06,Data Science and Optimization Adventures in Computational Biology and Medicine,NaN,NaN,True
29,330,2021-05-27,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,Australian National University,NaN,NaN,2021-05-27,Machine Learning and Optimization Endeavors in Computational Biology and Medicine,NaN,NaN,True
30,331,2021-10-14,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,"IDEXX Meeting, Boston University",NaN,NaN,2021-10-14,AI & ML Methods in Computational Biology and Medicine,NaN,NaN,True
31,332,2021-11-08,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,"Hariri Institute, Boston University",NaN,NaN,2021-11-08,The Role of Racial Disparities and Socioeconomic Factors in Health Predictive Modeling,NaN,NaN,True
32,333,2021-02-15,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,NSF Workshop on Predicting Pandemic Emergence,NaN,NaN,2021-02-15,Pandemic Predictive Analytics,NaN,NaN,True
36,337,2021-11-12,Scholarly presentation,Paschalidis I,NaN,NaN,NaN,NaN,"Innovation for Health Conference, IEEE Region 8",NaN,NaN,2021-11-12,"Robust Learning and Optimization in Computational Biology, Medicine, and Autonomy",NaN,NaN,True


In [ ]:
chatgpt[chatgpt['Part of FRP'] == True]